# Use SimSum Classification to Link FEBRL People Data

<a href="https://colab.research.google.com/github/rachhouse/intro-to-data-linking/blob/main/tutorial_notebooks/01_Link_FEBRL_Data_with_SimSum_Classification.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

In this tutorial, we'll link synthesized people datasets generated by the [Freely Extensible Biomedical Record Linkage (FEBRL)](https://sourceforge.net/projects/febrl/) project. The FEBRL-generated datasets represent cleaned datasets, so in this notebook, we will step through:
* data augmentation,
* blocking,
* comparing, and
* classification using the SimSum methodology.

## Google Colab Setup

In [1]:
# Check if we're running locally, or in Google Colab.
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    
# If we're running in Colab, download the tutorial functions file 
# to the Colab session local directory, and install required libraries.
if COLAB:
    import requests
    
    tutorial_functions_url = "https://raw.githubusercontent.com/rachhouse/intro-to-data-linking/main/tutorial_notebooks/linking_tutorial_functions.py"
    r = requests.get(tutorial_functions_url)
    
    with open("linking_tutorial_functions.py", "w") as fh:
        fh.write(r.text)
    
    !pip install -q recordlinkage jellyfish altair

## Imports

In [2]:
import itertools
import re

from typing import Dict, Tuple, Optional

import altair as alt
import jellyfish
import numpy as np
import pandas as pd
import recordlinkage as rl

# We have a couple helper functions from this file that we'll use for evaluation.
import linking_tutorial_functions as tutorial

## Define Filepaths

First, let's set up access to a few data resources that we'll need for the tutorial.

In [3]:
TRAINING_DATASET_A, TRAINING_DATASET_B, TRAINING_LABELS = tutorial.get_training_data_paths(COLAB)

## Load (Cleaned) Training Datasets

We'll load our training datasets into pandas DataFrames. We want to be able to take advantage of pandas indexing as we link our data (plus, the `recordlinkage` package that we'll be using later needs input DataFrames to be indexed by record id), so we'll set an index on each training DataFrame.

As mentioned above, we can consider the cleaning step of linking to be already done - the data generated by FEBRL is in a consistent format, and equivalent attributes have been encoded in the same manner for the two synthesized people datasets.

In [4]:
df_A = pd.read_csv(TRAINING_DATASET_A)
df_A = df_A.set_index("person_id_A")
df_A.head()

,first_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,age,phone_number,soc_sec_id
person_id_A,,,,,,,,,,,,
e21560e6-a7c8-4e25-afd8-7899d218dc58,dakota,geraghty,69,maclean street,skeers property,dandenong north,2529,nsw,19380417,31,03 01783133,6629995
f0b05811-de1f-4d18-b06e-2ab31a4c9671,james,colquhoun,118,conlon crescent,,birkdale,5043,nsw,19680112,,07 14327140,5350518
bac344e0-2c45-4a8e-ac3b-a2d8fa7e3a21,ruby,butt,103,,wollartukkee,east fremantle,4814,wa,19430120,30,02 88839517,3225206
771c6705-5a9d-4edc-bc22-fc6109f2b3dd,marcus,rees,5,charlick place,lindoran,ballarat,4216,nsw,,27,08 17239266,7355062
2230954b-45d3-4c77-9b1f-f9fd01ff7db7,jassim,belperio,36,john russell circuit,,eastwood,3131,nsw,19460129,20,02 61510457,9190750


In [5]:
df_B = pd.read_csv(TRAINING_DATASET_B)
df_B = df_B.set_index("person_id_B")
df_B.head()

,first_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,age,phone_number,soc_sec_id
person_id_B,,,,,,,,,,,,
d55bb330-3831-4d6d-82d9-041d5aa5a1da,charlotte,leukg,301,domain street,locn 1699,alma bay,2710,vic,,29,07 05109263,6356142
6cc556f2-6a5f-4293-9943-b2dfbd22e5b8,callie,heerscgap,23,dudi lzce,,mill park,2324,tas,19820623,9,02 82637596,6775114
1e08b6e1-db9d-47bc-a7e6-140ed8828d2f,alanx,nguyen,6,callaghan street,,albury,4575,nsw,19220115,27,08 82171717,5275665
295be7cd-e2a6-4ded-9ceb-23a4ab69dbb3,willjam,dud,83,purbrick street,glenveagh,muttabrra,6100,,19871212,23,07 54557966,7073899
e9438b01-a650-44b0-9762-d07f2c9dbd6a,lucy,baillie,34,hurley street,,glen iqnnes,5038,sa,19310448,,08 19431835,6880723


## Load Training Ground Truth Labels

One of the advantages of synthesized data, especially for tutorials and learning, is that we have ground truth labels for data. (This is rarely the case when you encounter linking problems in the wild). We'll load our known true links into a pandas DataFrame below.

In [6]:
df_ground_truth = pd.read_csv(TRAINING_LABELS)
df_ground_truth = df_ground_truth.set_index(["person_id_A", "person_id_B"])
df_ground_truth["ground_truth"] = df_ground_truth["ground_truth"].apply(lambda x: True if x == 1 else False)
df_ground_truth.head()

,,ground_truth
person_id_A,person_id_B,
b653e9aa-5c2b-4a9f-b9b5-a17569863be1,487a7c5f-5965-43ac-a9b5-1e0b2cb6c202,True
70da0560-1ff0-4972-9ab1-189f8d31795b,2559fa8c-b1c0-421b-92fd-e152cc9eafd1,True
69cdc37e-0d66-4ee7-8136-8a09484a8267,021ee521-6578-4d52-9a6e-b571c3840f34,True
95d331b4-d36f-4504-8135-6833e392c80b,4ed2d2c1-e70e-46b2-bade-c09d926f32c9,True
771ffc11-a39e-4820-a494-32b4838a350a,af5743f6-e95e-4f9d-a7e6-210177b50421,True


## Data Augmentation

Let's take a look at our data, and consider what we have currently available for blocking and comparing.

In [7]:
df_A.head(n=2)

,first_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,age,phone_number,soc_sec_id
person_id_A,,,,,,,,,,,,
e21560e6-a7c8-4e25-afd8-7899d218dc58,dakota,geraghty,69,maclean street,skeers property,dandenong north,2529,nsw,19380417,31,03 01783133,6629995
f0b05811-de1f-4d18-b06e-2ab31a4c9671,james,colquhoun,118,conlon crescent,,birkdale,5043,nsw,19680112,,07 14327140,5350518


It would probably make sense to block on people's first and last name, but, as we've noted, the realities of data entry typos, nicknames, aliases, OCR mishaps, and speech-to-text blips mean that using an exact blocker isn't going to work well. These fields are prime candidates for phonetic encoding!

We'll use the python [jellyfish library](https://pypi.org/project/jellyfish/) to encode our `first_name` and `surname` fields via two phonetic encoding algorithms, [**Soundex**](https://en.wikipedia.org/wiki/Soundex) and [**NYSIIS**](https://en.wikipedia.org/wiki/New_York_State_Identification_and_Intelligence_System).

We could also use a truncated exact blocking approach with the `soc_sec_id` field. For this, we'll create a new attribute containing the last three digits of the SSid.

And lastly, we'll cast the `date_of_birth` field to a pandas Timestamp field so that we can compare it more easily down the road.

In [8]:
def dob_to_date(dob: str) -> Optional[pd.Timestamp]:
    """ Transform string date in YYYYMMDD format to a pd.Timestamp.
        Return None if transformation is not successful.
    """
    date_pattern = r"(\d{4})(\d{2})(\d{2})"
    dob_timestamp = None
    
    try:
        m = re.match(date_pattern, dob.strip())
        if m:
            dob_timestamp = pd.Timestamp(int(m.group(1)), int(m.group(2)), int(m.group(3)))
    except:
        pass

    return dob_timestamp

In [9]:
%%time

for df in [df_A, df_B]:
    
    # Update NaNs to empty strings or jellyfish will choke.
    df["surname"] = df["surname"].fillna("")
    df["first_name"] = df["first_name"].fillna("")

    # Soundex phonetic encodings.
    df["soundex_surname"] = df["surname"].apply(lambda x: jellyfish.soundex(x))
    df["soundex_firstname"] = df["first_name"].apply(lambda x: jellyfish.soundex(x))
    
    # NYSIIS phonetic encodings.    
    df["nysiis_surname"] = df["surname"].apply(lambda x: jellyfish.nysiis(x))
    df["nysiis_firstname"] = df["first_name"].apply(lambda x: jellyfish.nysiis(x))
    
    # Last 3 of SSID.
    df["ssid_last3"] = df["soc_sec_id"].apply(lambda x: str(x)[-3:].zfill(3) if x else None)
    df["soc_sec_id"] = df["soc_sec_id"].astype(str)
    
    # DOB to date object.
    df["dob"] = df["date_of_birth"].apply(lambda x: dob_to_date(x))

CPU times: user 85.7 ms, sys: 3.28 ms, total: 89 ms
Wall time: 87.5 ms


Let's take a look at a sample of our new columns:

In [10]:
df_A.head(n=2)

,first_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,age,phone_number,soc_sec_id,soundex_surname,soundex_firstname,nysiis_surname,nysiis_firstname,ssid_last3,dob
person_id_A,,,,,,,,,,,,,,,,,,
e21560e6-a7c8-4e25-afd8-7899d218dc58,dakota,geraghty,69,maclean street,skeers property,dandenong north,2529,nsw,19380417,31,03 01783133,6629995,G623,D230,GARAGTY,DACAT,995,1938-04-17
f0b05811-de1f-4d18-b06e-2ab31a4c9671,james,colquhoun,118,conlon crescent,,birkdale,5043,nsw,19680112,,07 14327140,5350518,C425,J520,CALGAHAN,JAN,518,1968-01-12


## Blocking

Now that we've augmented our datasets, let's try some blocking! We'll use the python [`recordlinkage` library](https://github.com/J535D165/recordlinkage) for blocking. 

First, let's see how many candidate record pairs we would generate with a full blocker - meaning if we compared every record in dataset A to every record in dataset B. This produces the [Cartesian product](https://en.wikipedia.org/wiki/Cartesian_product) of the two datasets.

In [11]:
indexer = rl.Index()
indexer.add(rl.index.Full())

full_blocker_pairs = indexer.index(df_A, df_B)
max_candidate_record_pairs = full_blocker_pairs.shape[0]

print(f"{max_candidate_record_pairs:,} total pairs.")

25,000,000 total pairs.


`indexer.index` returns a pandas MultiIndex of the candidate record pairs: 

In [12]:
full_blocker_pairs

MultiIndex([('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ('e21560e6-a7c8-4e25-afd8-7899d218dc58', ...),
            ...
            ('81817521-6d45-4605-8052-b5fa370534a9', ...),
            ('81817521-6d45-4605-8052-b5fa370534a9', ...),
            ('81817521-6d45-4605-8052-b5fa370534a9', ...),
            ('81817521-6d45-4605-8052-b5fa370534a9', ...),
            ('81817521-6d45-4605-8052-b5fa370534a9', ...),
            ('81817521-6d45-4605-8052-b5fa370534a9', ...),
            ('81817521-6d45-4605-8052-b5

Even for very small datasets, like our training data, we're looking a huge amount of candidate record pairs to compare, unless we employ more selective blocking.

Recall that successful and efficient blocking minimizes:
* the quantity of generated candidate record pairs
* missed true links

So, first let's define a method which measures the percentage of true links captured by blocking, as well as the search space reduction.

In [13]:
def evaluate_blocking(
    max_candidate_pairs: int,
    candidate_pairs: pd.MultiIndex,
    df_true_links: pd.DataFrame
) -> Tuple[float, float]:
    """ Function to calculate blocking search space reduction and retained true links.
        Reports and returns search space reduction percentage and retained true links percentage.
    """
    
    # Calculate search space reduction.
    search_space_reduction = round((1 - (candidate_pairs.shape[0]/max_candidate_pairs)) * 100, 2)
    
    # Calculate retained true links percentage.
    total_true_links = df_true_links.shape[0]
    true_links_after_blocking = pd.merge(
        df_true_links,
        candidate_pairs.to_frame(),
        left_index=True,
        right_index=True,
        how="inner"
    ).shape[0]
    
    retained_true_link_percent = round((true_links_after_blocking/total_true_links) * 100, 2)

    
    print(f"{candidate_pairs.shape[0]:,} pairs after full blocking: {search_space_reduction}% search space reduction.")
    print(f"{retained_true_link_percent}% true links retained after full blocking.")
    
    return search_space_reduction, retained_true_link_percent 

We can evaluate the full blocker as such:

In [14]:
_, _ = evaluate_blocking(max_candidate_record_pairs, full_blocker_pairs, df_ground_truth)

25,000,000 pairs after full blocking: 0.0% search space reduction.
100.0% true links retained after full blocking.


This makes sense. If we use a full blocker, we won't have reduced our search space at all. And, since we consider every possible candidate pair, this will include all true links.

However, let's see if we can do better. Let's experiment with a few sets of different blockers.

In [15]:
indexer = rl.Index()

indexer.add(rl.index.Block("surname"))

candidate_pairs = indexer.index(df_A, df_B)

_, _ = evaluate_blocking(max_candidate_record_pairs, candidate_pairs, df_ground_truth)

79,991 pairs after full blocking: 99.68% search space reduction.
59.02% true links retained after full blocking.


In [16]:
indexer = rl.Index()

indexer.add(rl.index.Block("surname"))
indexer.add(rl.index.Block("first_name"))

candidate_pairs = indexer.index(df_A, df_B)

_, _ = evaluate_blocking(max_candidate_record_pairs, candidate_pairs, df_ground_truth)

166,074 pairs after full blocking: 99.34% search space reduction.
86.5% true links retained after full blocking.


In [17]:
indexer = rl.Index()

indexer.add(rl.index.Block("soundex_surname"))
indexer.add(rl.index.Block("soundex_firstname"))
indexer.add(rl.index.Block("nysiis_surname"))
indexer.add(rl.index.Block("nysiis_firstname"))

candidate_pairs = indexer.index(df_A, df_B)

_, _ = evaluate_blocking(max_candidate_record_pairs, candidate_pairs, df_ground_truth)

300,122 pairs after full blocking: 98.8% search space reduction.
93.04% true links retained after full blocking.


In [18]:
indexer = rl.Index()

indexer.add(rl.index.Block("soundex_surname"))
indexer.add(rl.index.Block("soundex_firstname"))
indexer.add(rl.index.Block("nysiis_surname"))
indexer.add(rl.index.Block("nysiis_firstname"))
indexer.add(rl.index.Block("ssid_last3"))
indexer.add(rl.index.Block("date_of_birth"))

candidate_pairs = indexer.index(df_A, df_B)

_, _ = evaluate_blocking(max_candidate_record_pairs, candidate_pairs, df_ground_truth)

653,588 pairs after full blocking: 97.39% search space reduction.
100.0% true links retained after full blocking.


## Comparing

After we're reasonably satisifed with our blockers, we can move on to comparing our candidate record pairs. Recall that in the comparison step, for each candidate record pair, we compare their attributes to generate a comparison vector. Once again, we'll use [`recordlinkage`](https://github.com/J535D165/recordlinkage) to define our comparators. `recordlinkage` offers a variety of built-in comparators to use for string, numeric, and datetime fields.

* We can use exact comparators for our phonetic encoding fields.
* We'll use Jaro-Winkler comparison for the name fields, as this comparison approach is specifically designed for comparison of names.
* For the other string fields, we'll opt for Damerau-Levenshtein, which does a nice job in accomodating data entry typos.
* For the DOB, we'll use a date comparison.

In [19]:
%%time

comparer = rl.Compare()

# Phonetic encodings.
comparer.add(rl.compare.Exact("soundex_surname", "soundex_surname", label="soundex_surname"))
comparer.add(rl.compare.Exact("soundex_firstname", "soundex_firstname", label="soundex_firstname"))
comparer.add(rl.compare.Exact("nysiis_surname", "nysiis_surname", label="nysiis_surname"))
comparer.add(rl.compare.Exact("nysiis_firstname", "nysiis_firstname", label="nysiis_firstname"))

# First & last name.
comparer.add(rl.compare.String("surname", "surname", method="jarowinkler", label="last_name"))
comparer.add(rl.compare.String("first_name", "first_name", method="jarowinkler", label="first_name"))

# Address.
comparer.add(rl.compare.String("address_1", "address_1", method="damerau_levenshtein", label="address_1"))
comparer.add(rl.compare.String("address_2", "address_2", method="damerau_levenshtein", label="address_2"))
comparer.add(rl.compare.String("suburb", "suburb", method="damerau_levenshtein", label="suburb"))
comparer.add(rl.compare.String("postcode", "postcode", method="damerau_levenshtein", label="postcode"))
comparer.add(rl.compare.String("state", "state", method="damerau_levenshtein", label="state"))

# Other fields.
comparer.add(rl.compare.Date("dob", "dob", label="date_of_birth"))
comparer.add(rl.compare.String("phone_number", "phone_number", method="damerau_levenshtein", label="phone_number"))
comparer.add(rl.compare.String("soc_sec_id", "soc_sec_id", method="damerau_levenshtein", label="ssn"))

features = comparer.compute(candidate_pairs, df_A, df_B)

CPU times: user 45.4 s, sys: 811 ms, total: 46.2 s
Wall time: 45.8 s


You can see that the output of the compare step is a collection of comparison/feature vectors, one for each candidate record pair. `recordlinkage` returns these vectors as a pandas Dataframe, indexed on the record pair ids.

In [20]:
features

soundex_surname  \
person_id_A                          person_id_B                                             
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                0   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                0   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5                0   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f                0   
                                     201c4dba-825a-42f2-b7a8-832b792af90b                0   
...                                                                                    ...   
ffe6d0b1-5313-411b-89d7-77694d9cd615 f324d96d-c264-4b87-87f5-dc4a81aaed2e                1   
                                     f8253878-b5b9-4f5c-b71d-3ee03555ca5b                0   
                                     faa3262c-1485-429f-a2e8-ff56b5de5cee                0   
                                     fb0dc9ce-e22d-4731-8181-3cdf5b144d69                0   
                                     fc7a3f24-473a-41da-b9bc-9fdec5c05292                0   

                                                                           soundex_firstname  \
person_id_A                          person_id_B                                               
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                  1   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                  1   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5                  1   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f                  1   
                                     201c4dba-825a-42f2-b7a8-832b792af90b                  0   
...                                                                                      ...   
ffe6d0b1-5313-411b-89d7-77694d9cd615 f324d96d-c264-4b87-87f5-dc4a81aaed2e                  0   
                                     f8253878-b5b9-4f5c-b71d-3ee03555ca5b                  1   
                                     faa3262c-1485-429f-a2e8-ff56b5de5cee                  0   
                                     fb0dc9ce-e22d-4731-8181-3cdf5b144d69                  1   
                                     fc7a3f24-473a-41da-b9bc-9fdec5c05292                  0   

                                                                           nysiis_surname  \
person_id_A                          person_id_B                                            
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82               0   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc               0   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5               0   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f               0   
                                     201c4dba-825a-42f2-b7a8-832b792af90b               0   
...                                                                                   ...   
ffe6d0b1-5313-411b-89d7-77694d9cd615 f324d96d-c264-4b87-87f5-dc4a81aaed2e               1   
                                     f8253878-b5b9-4f5c-b71d-3ee03555ca5b               0   
                                     faa3262c-1485-429f-a2e8-ff56b5de5cee               0   
                                     fb0dc9ce-e22d-4731-8181-3cdf5b144d69               0   
                                     fc7a3f24-473a-41da-b9bc-9fdec5c05292               0   

                                                                           nysiis_firstname  \
person_id_A                          person_id_B                                              
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                 1   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                 1   
             

Here's a look at an individual comparison vector:

In [21]:
display(features.iloc[0].name)
display(features.iloc[0])

('002cf4ec-57d0-4ebf-a31b-88db4441ff2e',
 '061b9c3f-afbe-41e5-923b-3de29a4e5b82')

soundex_surname      0.000000
soundex_firstname    1.000000
nysiis_surname       0.000000
nysiis_firstname     1.000000
last_name            0.000000
first_name           1.000000
address_1            0.263158
address_2            0.225806
suburb               0.384615
postcode             0.200000
state                0.250000
date_of_birth        0.000000
phone_number         0.333333
ssn                  0.000000
Name: (002cf4ec-57d0-4ebf-a31b-88db4441ff2e, 061b9c3f-afbe-41e5-923b-3de29a4e5b82), dtype: float64

## Add labels to feature vectors

We've generated our comparison/feature vectors, now we're ready to classify! To begin, we'll add our ground truth labels to the features DataFrame. Note that `df_ground_truth` just contains the true links, so we'll use a left join and then `fillna` with `False` for any records that are not true links.

In [22]:
df_labeled_features = pd.merge(
    features,
    df_ground_truth,
    on=["person_id_A", "person_id_B"],
    how="left"
)

df_labeled_features["ground_truth"].fillna(False, inplace=True)
df_labeled_features.head()

soundex_surname  \
person_id_A                          person_id_B                                             
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                0   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                0   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5                0   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f                0   
                                     201c4dba-825a-42f2-b7a8-832b792af90b                0   

                                                                           soundex_firstname  \
person_id_A                          person_id_B                                               
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                  1   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                  1   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5                  1   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f                  1   
                                     201c4dba-825a-42f2-b7a8-832b792af90b                  0   

                                                                           nysiis_surname  \
person_id_A                          person_id_B                                            
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82               0   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc               0   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5               0   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f               0   
                                     201c4dba-825a-42f2-b7a8-832b792af90b               0   

                                                                           nysiis_firstname  \
person_id_A                          person_id_B                                              
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                 1   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                 1   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5                 1   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f                 1   
                                     201c4dba-825a-42f2-b7a8-832b792af90b                 0   

                                                                           last_name  \
person_id_A                          person_id_B                                       
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82   0.000000   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc   0.577778   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5   0.588889   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f   0.455556   
                                     201c4dba-825a-42f2-b7a8-832b792af90b   0.577778   

                                                                           first_name  \
person_id_A                          person_id_B                                        
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82    1.000000   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc    1.000000   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5    1.000000   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f    1.000000   
                                     201c4dba-825a-42f2-b7a8-832b792af90b    0.611111   

                                                                           address_1  \
person_id_A                          person_id_B                                       
002cf4ec

## Calculate SimSum Scores

Once again, SimSum is the simplest approach to linking classification. To generate our scores for the candidate record pairs, we simply sum the values each attribute comparison score into a single score for each record.

In [23]:
df_labeled_features["simsum"] = df_labeled_features.drop("ground_truth", axis=1).sum(axis=1)
df_labeled_features.head()

soundex_surname  \
person_id_A                          person_id_B                                             
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                0   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                0   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5                0   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f                0   
                                     201c4dba-825a-42f2-b7a8-832b792af90b                0   

                                                                           soundex_firstname  \
person_id_A                          person_id_B                                               
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                  1   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                  1   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5                  1   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f                  1   
                                     201c4dba-825a-42f2-b7a8-832b792af90b                  0   

                                                                           nysiis_surname  \
person_id_A                          person_id_B                                            
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82               0   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc               0   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5               0   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f               0   
                                     201c4dba-825a-42f2-b7a8-832b792af90b               0   

                                                                           nysiis_firstname  \
person_id_A                          person_id_B                                              
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82                 1   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc                 1   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5                 1   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f                 1   
                                     201c4dba-825a-42f2-b7a8-832b792af90b                 0   

                                                                           last_name  \
person_id_A                          person_id_B                                       
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82   0.000000   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc   0.577778   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5   0.588889   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f   0.455556   
                                     201c4dba-825a-42f2-b7a8-832b792af90b   0.577778   

                                                                           first_name  \
person_id_A                          person_id_B                                        
002cf4ec-57d0-4ebf-a31b-88db4441ff2e 061b9c3f-afbe-41e5-923b-3de29a4e5b82    1.000000   
                                     081ec178-99a1-4895-b96e-7c03cf8bbfdc    1.000000   
                                     17b274b5-aa3f-43cc-96ae-21283b7d1ca5    1.000000   
                                     1f70d4cd-3106-4d15-af9f-1617a43ca83f    1.000000   
                                     201c4dba-825a-42f2-b7a8-832b792af90b    0.611111   

                                                                           address_1  \
person_id_A                          person_id_B                                       
002cf4ec

## Choosing a SimSum Classification Threshold

Now that we've generated scores for all of our candidate record pairs, the next step is to determine a threshold at which we can classify a record pair as a link, or not-a-link. To do this, it's first helpful to look at the score distribution.

### "Model" Score Distribution

We can see a pretty clear boundary between not-links and links when it comes to the SimSum score. There's a bit of an overlap from 7 - 9.5, but it looks like we'll probably want to set the cutoff somewhere in that range.

In [24]:
tutorial.plot_model_score_distribution(
    df_labeled_features,
    score_column_name="simsum",  
)

alt.Chart(...)

### Precision and Recall at Varying Thresholds

Next, we'll take a look at the calculated precision and recall at varying model score thresholds. Below is a function which calculates precision and recall for a range of scores, and which also returns the top k and bottom k-scoring links for our inspection.

In [25]:
def evaluate_linking(
    df: pd.DataFrame,
    df_true_links: pd.DataFrame,
    df_left: pd.DataFrame,
    df_right: pd.DataFrame,
    score_column_name: Optional[str] = "score",
    ground_truth_column_name: Optional[str] = "ground_truth",
    k: int = 10
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """ Use model results to calculate precision & recall metrics,
        top k links, and bottom k links.
    
        Args:
            df: dataframe containing model scores, and ground truth labels
                indexed on df_left index, df_right index
            df_true_links: dataframe containing true links,
                indexed on df_left index, df_right index
            df_left: dataframe containing attributes for "left"-linked entities
            df_right: dataframe containing attributes for "right"-linked entities
            score_column_name: Optional string name of column containing model scores 
            ground_truth_column_name: Optional string name of column containing ground
                truth values
                
        Returns:
            Tuple containing:
                pandas dataframe with precision and recall evaluation data
                pandas dataframe with top k scoring links
                pandas dataframe with bottom k scoring links
    """    
    # Calculate eval data at threshold intervals from zero to max score. 
    # Max score is generally 1.0 if using a ML model, but with SimSum it
    # can get much larger.
    eval_data = []
    max_score = max(1, max(df[score_column_name]))

    for threshold in np.linspace(0, max_score, 50):
        tp = df[(df[score_column_name] >= threshold) & (df[ground_truth_column_name] == True)].shape[0]
        fp = df[(df[score_column_name] >= threshold) & (df[ground_truth_column_name] == False)].shape[0]
        tn = df[(df[score_column_name] < threshold) & (df[ground_truth_column_name] == False)].shape[0]
        fn = df[(df[score_column_name] < threshold) & (df[ground_truth_column_name] == True)].shape[0]
        
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * ((precision * recall)/(precision + recall))
        
        eval_data.append(
            {
                "threshold" : threshold,
                "tp" : tp,
                "fp" : fp,
                "tn" : tn,
                "fn" : fn,
                "precision" : precision,
                "recall" : recall,
                "f1" : f1
            }
        )

    # Assemble the top and bottom k links (sorted by model score).
    # This is done by sorting the model results frame by score, capping rows at k,
    # and then joining the original link entity data via the dataframe indices.
    # This gives us the model score as well as the actual human-readable attributes
    # for each link.
    def _join_original_entity_data_to_links(
        df_k_links: pd.DataFrame,
        df_left: pd.DataFrame,
        df_right: pd.DataFrame
    ) -> pd.DataFrame:
        """Helper function to join entity data to a dataframe of link results."""
        
        # Join data from left entities.
        df_k_links = pd.merge(
            df_k_links,
            df_left,
            left_on=df_left.index.name,
            right_index=True,
        )
        
        # Join data from right entities.
        return pd.merge(
            df_k_links,
            df_right,
            left_on=df_right.index.name,
            right_index=True,
        )  
    
    df_top_k_links = _join_original_entity_data_to_links(
        df[[score_column_name, ground_truth_column_name]].sort_values(score_column_name, ascending=False).head(n=k).reset_index(),
        df_left,
        df_right
    )
    
    df_bottom_k_links = _join_original_entity_data_to_links(
        df[[score_column_name, ground_truth_column_name]].sort_values(score_column_name).head(n=k).reset_index(),
        df_left,
        df_right    
    )
    
    return pd.DataFrame(eval_data), df_top_k_links, df_bottom_k_links

In [26]:
df_eval, df_top_links, df_bottom_links = evaluate_linking(
    df=df_labeled_features,
    df_true_links=df_ground_truth,
    df_left=df_A,
    df_right=df_B,
    score_column_name = "simsum",
)

In [27]:
df_eval.head()

,threshold,tp,fp,tn,fn,precision,recall,f1
0,0.000000,5000,648588,0,0,0.007650,1.0,0.015184
1,0.285714,5000,648588,0,0,0.007650,1.0,0.015184
2,0.571429,5000,648588,0,0,0.007650,1.0,0.015184
3,0.857143,5000,648584,4,0,0.007650,1.0,0.015184
4,1.142857,5000,648390,198,0,0.007652,1.0,0.015189


The plot precision and recall at varying score thresholds reinforces what we noted earlier in the score distribution - that our most suitable cutoff is in the range of 7 to 9.5. It relies on your own particular use case to determine exactly where the cutoff should be set (e.g. Is recall more important than precision, or vice versa?).

In [28]:
tutorial.plot_precision_recall_vs_threshold(df_eval)

alt.Chart(...)

### Top Scoring `k` Links

Another way to gain insight into the performance of link classification is examining individual links in score ranges of interest. Below, we've captured the top 10 and bottom 10 scoring links for inspection, as well as their original attribute values.

In [29]:
display_cols = [
    "first_name", "surname",
    "street_number", "address_1", "address_2", "suburb", "postcode", "state",
    "date_of_birth", "age", "phone_number", "soc_sec_id",
    "soundex_surname", "soundex_firstname",
    "nysiis_surname", "nysiis_firstname",
]

display_cols = [[f"{col}_x", f"{col}_y"] for col in display_cols]
display_cols = list(itertools.chain.from_iterable(display_cols))

In [30]:
with pd.option_context('display.max_columns', None):
    display(df_top_links[["person_id_A", "person_id_B", "simsum", "ground_truth"] + display_cols])

,person_id_A,person_id_B,simsum,ground_truth,first_name_x,first_name_y,surname_x,surname_y,street_number_x,street_number_y,address_1_x,address_1_y,address_2_x,address_2_y,suburb_x,suburb_y,postcode_x,postcode_y,state_x,state_y,date_of_birth_x,date_of_birth_y,age_x,age_y,phone_number_x,phone_number_y,soc_sec_id_x,soc_sec_id_y,soundex_surname_x,soundex_surname_y,soundex_firstname_x,soundex_firstname_y,nysiis_surname_x,nysiis_surname_y,nysiis_firstname_x,nysiis_firstname_y
0,c545a5c2-8acf-40f7-b18a-103edc6fdef5,1e08f066-d2af-49e1-b571-ac60216b480f,14.0,True,teneille,teneille,hand,hand,49,4,cutbush street,cutbush street,,,woodpark,woodpark,2023,2023,qld,qld,19140602,19140602,27,77,08 90051333,08 90051333,4073741,4073741,H530,H530,T540,T540,HAD,HAD,TANAL,TANAL
1,ad506114-453b-40db-b836-7f33df002c55,d36f2e13-26c7-4c62-a26d-e45c396a2da6,14.0,True,olivia,olivia,beams,beams,28,61,bingara place,bingara place,,,pymble,pymble,3909,3909,nsw,nsw,19520717,19520717,23,23,04 14201344,04 14201344,2202477,2202477,B520,B520,O410,O410,BAN,BAN,OLAV,OLAV
2,b8046dea-c215-4939-975f-5b949e460ab0,0cd60f41-d51b-4952-b1c0-d190585ee67b,14.0,True,joshua,joshua,matthews,matthews,7,32,watts street,watts street,,,southbank,southbank,3340,3340,nsw,nsw,19521122,19521122,28,,07 89548454,07 89548454,3331727,3331727,M320,M320,J200,J200,MATAE,MATAE,JAS,JAS
3,3a669033-9075-4732-96ba-777b95d8b880,9bb21b88-03b5-4805-8e78-435d59da2f5c,14.0,True,katelyn,katelyn,rees,rees,23,23,alpen street,alpen street,springdale,springdale,port douglas,port douglas,6018,6018,nsw,nsw,19740302,19740302,26,27,08 60686489,08 60686489,6593638,6593638,R200,R200,K345,K345,R,R,CATALYN,CATALYN
4,c0fc77ed-9d7c-4ee7-9042-d5cfb7bb61da,e3b3662f-0a2d-4d21-b496-43bbd4747768,14.0,True,ned,ned,kammermann,kammermann,67,67,glasgow place,glasgow place,,,beeliar,beeliar,2193,2193,qld,qld,19840928,19840928,22,29,08 26074713,08 26074713,9085870,9085870,K565,K565,N300,N300,CANARNAN,CANARNAN,NAD,NAD
5,0ee23d3c-cd5e-4164-af2d-355c2fe4d4b0,10310a0a-f40b-4093-96f8-da691b9721b9,14.0,True,jack,jack,dunstone,dunstone,44,45,hingston close,hingston close,mount tongbong,mount tongbong,belmont,belmont,2541,2541,nsw,nsw,19960316,19960316,38,36,04 50838118,04 50838118,3057451,3057451,D523,D523,J200,J200,DANSTAN,DANSTAN,JAC,JAC
6,013f16ad-e627-4fec-9551-5b8cd22f935e,1706f8d0-eca5-490e-a340-078bc801f5b0,14.0,True,olivia,olivia,hassall,hassall,2,1,paech place,paech place,,,deepwater,deepwater,2283,2283,nsw,nsw,19790514,19790514,24,,02 58606717,02 58606717,1599081,1599081,H240,H240,O410,O410,HASAL,HASAL,OLAV,OLAV
7,21f7e8dd-28d4-4322-be5b-91fc6d7b16d3,0e2ad218-3cc4-411b-9947-6de0cd79b7f6,14.0,True,lauren,lauren,hanks,hanks,3,169,la perouse street,la perouse street,macovin ridge,macovin ridge,raglan,raglan,7009,7009,nsw,nsw,19660326,19660326,37,37,07 38077292,07 38077292,4288465,4288465,H520,H520,L650,L650,HANC,HANC,LARAN,LARAN
8,b73dc685-bb48-4e94-a889-f694a4f415f4,e24d894a-6d63-4066-9ec5-2ca204eb1efe,14.0,True,shenae,shenae,lowe,lowe,106,1050,florence fuller crescent,florence fuller crescent,,,bassendean,bassendean,2577,2577,vic,vic,19750221,19750221,9,30,08 97591633,08 97591633,2630839,2630839,L000,L000,S500,S500,LAO,LAO,SAN,SAN
9,49d22fd4-764f-4082-b6df-b279450a0c8c,58ea2735-398d-40a4-b48f-51a1733b65db,14.0,True,james,james,morrison,morrison,11,15,ingram street,ingram street,villa 2,villa 2,noble park,noble park,2148,2148,nsw,nsw,19810409,19810409,30,,04 07562543,04 07562543,3227052,3227052,M625,M625,J520,J520,MARASAN,MARASAN,JAN,JAN


### Bottom Scoring `k` Links

In [31]:
with pd.option_context('display.max_columns', None):
    display(df_bottom_links[["person_id_A", "person_id_B", "simsum", "ground_truth"] + display_cols])

,person_id_A,person_id_B,simsum,ground_truth,first_name_x,first_name_y,surname_x,surname_y,street_number_x,street_number_y,address_1_x,address_1_y,address_2_x,address_2_y,suburb_x,suburb_y,postcode_x,postcode_y,state_x,state_y,date_of_birth_x,date_of_birth_y,age_x,age_y,phone_number_x,phone_number_y,soc_sec_id_x,soc_sec_id_y,soundex_surname_x,soundex_surname_y,soundex_firstname_x,soundex_firstname_y,nysiis_surname_x,nysiis_surname_y,nysiis_firstname_x,nysiis_firstname_y
0,d480a5e3-95bc-406f-98e4-e3d21099c8a1,0bd1b934-25a5-41cc-a4f8-00cf9a46cc0a,0.714534,False,nicholas,jayb,rees,humphfcys,34,32,,higgerson street,,windsor dental centre,mitcham,balwyn north,2190,4802,nsw,wa,,,31,21,03 48152407,,8000601,3725158,R200,H512,N242,J100,R,HANFCY,NACAL,JAYB
1,2341aeb6-8542-4cb4-9a95-fc38a053bef7,a31b3b72-f66f-4a63-9e01-2b303c2d94ac,0.740559,False,gabriel,joxhua,filipov,prodw,56,173,bavin street,,dudley specialist medical centre,,elwood,mona vale,7008,2672,nsw,,,,37,29,07 44471940,,6353487,1978244,F411,P630,G164,J200,FALAPAV,PRADW,GABRAL,JAX
6,d5cda8b4-0be9-495a-9dd4-3c299a30fe73,a31b3b72-f66f-4a63-9e01-2b303c2d94ac,0.876923,False,brinley,joxhua,millar,prodw,74,173,shumack street,,kerry street,,dalby,mona vale,3805,2672,vic,,,,28,29,03 29449716,,5414901,1978244,M460,P630,B654,J200,MALAR,PRADW,BRANLY,JAX
9,8b733653-215b-4987-b852-318f55c833de,a31b3b72-f66f-4a63-9e01-2b303c2d94ac,0.888763,False,emiily,joxhua,hammer,prodw,16,173,neumayer street,,rp 31513,,leichhardt,mona vale,4500,2672,vic,,,,34,29,07 75170221,,4013468,1978244,H560,P630,E540,J200,HANAR,PRADW,ENALY,JAX
2,dc78c431-ff85-4fa9-8771-53cec35d5d13,6a607b27-92ec-416e-af50-2e670d43ef24,0.842857,False,hollie,zac,woodbury,canini,86,1716,,whalan lace,,oxford,south perth,terreyhills,3143,4270,wa,nsw,,,37,32,,04 40897322,8392168,9700884,W316,C550,H400,Z200,WADBARY,CANAN,HALY,ZAC
3,57e9e850-9a31-463a-941e-562071e34001,92c60ebf-4b93-4462-a5c7-fd342d0f6f22,0.848352,False,xani,declen,ponter,kiss,52,33,,flecker place,inglewood,warra warra,campbelltown,bonny hills,4421,5052,nsw,qlc,,,13,25,07 68697185,,6980982,9377051,P536,K200,X500,D245,PANTAR,C,XAN,DACLAN
4,477db918-ca95-4c5f-a4b4-c655552151dd,781435b9-562a-4ddc-aa7c-981b107addfd,0.859524,False,jordan,mitchell,,koolen,10,13,valder place,crowleyndourt,,upper meroo,winmalee,bray park,5049,4814,qld,,,,29,29,,04 66042159,5686855,2748162,,K450,J635,M324,,CALAN,JARDAN,MATCAL
5,da078a04-d1bb-42f9-901c-d5c5f58c1d60,5858710e-7b10-4f8e-956b-767da789d01e,0.866667,False,james,ruby,miles,rafandlli,29,35,churchill way,elliott street,little glencoe,,peregian beach,ballarat,6230,4511,nsw,,,,24,,08 21445124,,7227769,6348843,M420,R153,J520,R100,MAL,RAFANDL,JAN,RABY
7,92a3982c-d0e2-4cb0-8899-7377f48aa348,aee7064f-b8c9-4fb6-93f8-74d3b6823f51,0.886447,False,emiily,,dixon,blake,9,8,antill street,clement zlace,mcivor house,,botany,mermaid waters,4560,5097,vic,,,,22,31,07 27293537,,4367087,5274614,D250,B420,E540,,DAXAN,BLAC,ENALY,
8,9cbbd4a0-b5db-4c01-b51b-68565d3d5d49,2682766c-cf07-43b4-be07-8d71a6f002e3,0.888095,False,connor,asha,rees,gao,17,15,bugden avenue,,,the meadows,guyra,brookvale,3040,5167,vic,nds,,,,,,04 63180654,8638037,1590501,R200,G000,C560,A200,R,G,CANAR,AS
